In [147]:
import pandas as pd
import re
from datetime import datetime
from fetch_process_function import calculate_pnl
'''
不用看，测试文件
data_process.ipynb和data_fetch.ipynb是获取数据以及处理的一个流程，和cal_pnL.ipynb，fetch_process_function.py是同样的作用，后者是将前者的代码整理成一个py文件，方便调用，更加简洁
'''
OUTPUT_FILE = 'C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result'
api_key = "VQAIR728IM4Z8RZKPYBR4ESM5I3WBZK2C1" # my free API key, you can get one at https://etherscan.io/myapikey
base_url = "https://api.etherscan.io/v2/api" # We're using the v2 API 2024/12/12
ADDRESS = "0x2c89a2ee92b9870f55989b4132a58c0e85222d86" # Address of the contract we want to get the source code of
INVALID_ADDRESS = "0x0000000000000000000000000000000000000000"

In [148]:
def combine_group(group):
    # 1.检查 timestamp 是否唯一
    if group['timeStamp'].nunique() != 1:
        print("❌ The timestamps are not consistent.")
        return None

    # 2.检查 from 和 to 地址是否只有两种，且数量相同
    from_addresses = group['from'].unique()
    to_addresses = group['to'].unique()
    if len(from_addresses) != 2 or len(to_addresses) != 2:
        print("❌ The from/to addresses are not exactly two.")
        return None

    if group['from'].value_counts().values.tolist() != group['to'].value_counts().values.tolist():
        print("❌ The counts of from/to addresses do not match.")
        return None

    # 3.检查 token 是否只有两种
    if group['tokenSymbol'].nunique() != 2:
        print("❌ The tokens are not exactly two.")
        return None

    dateTime = group['dateTime'].iloc[0]
    blockNumber = group['blockNumber'].iloc[0]
    timeStamp = group['timeStamp'].iloc[0]
    hash_value = group['hash'].iloc[0]

    # 按 tokenSymbol 和 from 地址汇总 value
    # 这里没有考虑顺序，因为只要进出对应没错的话都不影响计算计算pnl
    token_values = group.groupby(['tokenSymbol', 'from'])['ActualValue'].sum().reset_index()

    unique_token_symbols = group['tokenSymbol'].drop_duplicates().tolist()
    token_values['tokenSymbol'] = pd.Categorical(
        token_values['tokenSymbol'], categories=unique_token_symbols, ordered=True
    )
    token_values = token_values.sort_values(by='tokenSymbol').reset_index(drop=True)

    # 生成结果 DataFrame
    combined_df = pd.DataFrame({
        'dateTime': [dateTime, dateTime],
        'blockNumber': [blockNumber, blockNumber],
        'timeStamp': [timeStamp, timeStamp],
        'hash': [hash_value, hash_value],
        'from': [token_values['from'].iloc[1], token_values['from'].iloc[0]],
        'to': [group['to'].iloc[1], group['to'].iloc[0]],  # 对应流入和流出
        'ActualValue': [token_values['ActualValue'].iloc[1], token_values['ActualValue'].iloc[0]],
        'tokenName': [group['tokenName'].iloc[1], group['tokenName'].iloc[0]],
        'tokenSymbol': [token_values['tokenSymbol'].iloc[1], token_values['tokenSymbol'].iloc[0]]
    })

    return combined_df


def highlight_three_records(grouped_df):
    matched_hashes = []
    combined_hashes_list = []
    for tx_hash, group in grouped_df:
        if len(group) >= 3:
            print(f"⚠️ High Alert: Transaction Hash {tx_hash} contains {len(group)} records:")
            matched_hashes.append(tx_hash)
            
            # 1.先检查是不是偶数个 奇数个显然不合理
            if len(group) % 2 != 0:
                print("❌ The number of records is not even.")
            else:
                # 调用 combine_group 检查并合并
                combined_hash = combine_group(group)
                if combined_hash is not None:
                    combined_hashes_list.append(combined_hash)
                    
    if combined_hashes_list:
        combined_hashes_df = pd.concat(combined_hashes_list, ignore_index=True)
    else:
        combined_hashes_df = pd.DataFrame()
    return matched_hashes, combined_hashes_df


In [149]:
global BASE_TOKENS
BASE_TOKENS = {"USDT", "USDC", "USDE"}
base_pattern = re.compile('|'.join(BASE_TOKENS))
ADDRESS = ADDRESS.lower()

csv_file = 'C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result/erc20_transfers.csv'
transaction_data = pd.read_csv(csv_file)
transaction_data['dateTime'] = pd.to_datetime(transaction_data['dateTime'])
transaction_data.loc[:, 'ActualValue'] = pd.to_numeric(transaction_data['ActualValue'], errors='coerce')
# 如果不转换为数据的话计算会出错
transaction_data = transaction_data.sort_values(by=['dateTime', 'hash']).reset_index(drop=True)
# make sure the date data is correct

columns_to_keep = [
    'dateTime', 'blockNumber', 'timeStamp', 'hash', 'from', 'to',
    'ActualValue', 'tokenName', 'tokenSymbol'
    ]
filtered_transaction_data = transaction_data[columns_to_keep]
filtered_transaction_data.to_csv('filtered_transaction_data.csv', index=False)
# 清洗数据
filtered_transaction_data = filtered_transaction_data[
    (transaction_data['from'] != INVALID_ADDRESS) &
    (transaction_data['to'] != INVALID_ADDRESS)
]
output_file = 'cleaned_transaction_data.csv'
filtered_transaction_data[columns_to_keep].to_csv(output_file, index=False)

In [150]:
duplicate_hashes = filtered_transaction_data[filtered_transaction_data.duplicated(subset=['hash'], keep=False)]

# check if there are abnoraml data, print it and delete from original data
three_record_hashes, combined_df = highlight_three_records(duplicate_hashes.groupby('hash'))

# delete the dulicate data that are more than 2, so that we can cntinue the calculation
duplicate_hashes = duplicate_hashes[~duplicate_hashes['hash'].isin(three_record_hashes)]

merged_df = pd.concat([duplicate_hashes, combined_df], ignore_index=True)
merged_df = merged_df.sort_values(by='timeStamp').reset_index(drop=True)
merged_df.to_csv('merged_transaction_data.csv', index=False)

⚠️ High Alert: Transaction Hash 0x0fcc53040dfc925d1b169fbedc575f9eec30fad4133bdd38ed8efdfc3f2969d3 contains 6 records:
⚠️ High Alert: Transaction Hash 0x1f4ca78710717a409b2cee42702db582b13d4e544adf116a86c605f92973d935 contains 6 records:
⚠️ High Alert: Transaction Hash 0x2b198f8b99cc4a183fdb5125b09e1f32521b7f5351e6b12307d5022ba030708c contains 4 records:
⚠️ High Alert: Transaction Hash 0x3de619b29ddab8d241f47d7a21bde8318691b71244e4658dedf97f5eba4705b3 contains 4 records:
⚠️ High Alert: Transaction Hash 0x3f1e3bfa355baded87207a2de30165c09a64d1ecb070f04c819587cc9386a1b0 contains 4 records:
⚠️ High Alert: Transaction Hash 0x3f550a50553c3033b0b0d32a435d1c0a9d3ba8323a4f9a8de1ffdcaa2cbaf739 contains 20 records:
⚠️ High Alert: Transaction Hash 0x427a515c5b0967335ebc0dc437bdc75ca1d2cf6d5573a6e477b399de40dcb9a6 contains 4 records:
⚠️ High Alert: Transaction Hash 0x52dd542f161f822026ef0050e6002c161957d086113dd7e5bf23a6e54ee02074 contains 6 records:
⚠️ High Alert: Transaction Hash 0x53d7b42ca96ea

In [146]:
output_records = []

for hash_val, group in duplicate_hashes.groupby('hash'):
    # base_tokens = group[group['tokenSymbol'].isin(BASE_TOKENS)]
    # other_tokens = group[~group['tokenSymbol'].isin(BASE_TOKENS)]
    base_tokens = group[group['tokenSymbol'].str.contains(base_pattern)]
    other_tokens = group[~group['tokenSymbol'].str.contains(base_pattern)]
    
    if not base_tokens.empty and not other_tokens.empty:
        if base_tokens['to'].iloc[0] == ADDRESS:
            transaction_type = "SELL"
        else:
            transaction_type = "BUY"

        base_token_info = f"{base_tokens['ActualValue'].sum()} {base_tokens['tokenSymbol'].iloc[0]}"
        other_token_info = f"{other_tokens['ActualValue'].sum()} {other_tokens['tokenSymbol'].iloc[0]}"

        record = f"{group['timeStamp'].iloc[0]} W {transaction_type} {other_token_info} of {base_token_info} (at {group['dateTime'].iloc[0]})"
        output_records.append({
            "formatted_record": record,
            "timeStamp": group['timeStamp'].iloc[0],
            "dateTime": group['dateTime'].iloc[0]
        })
    elif base_tokens.empty and not other_tokens.empty:
        # 如果 tokenSymbol 都不在 BASETOKENS 中，额外输出一下
        record = f"{group['timeStamp'].iloc[0]} Same Hash {hash_val}: Tokens not in BASE_TOKENS: {', '.join(group['tokenSymbol'].unique())} (at {group['dateTime'].iloc[0]})"
        output_records.append({
            "formatted_record": record,
            "timeStamp": group['timeStamp'].iloc[0],
            "dateTime": group['dateTime'].iloc[0]
        })
        
output_df = pd.DataFrame(output_records)
if not output_df.empty:
    output_df = output_df.sort_values(by='timeStamp').reset_index(drop=True)

    print("\nFormatted Transactions:")
    for record in output_df['formatted_record']:
        print(record)



Formatted Transactions:
1732901915 W BUY 1.70948726 cbBTC of 166132.207127 USDC (at 2024-11-29 17:38:35)
1732901939 W BUY 3.76996489 cbBTC of 366366.941125 USDC (at 2024-11-29 17:38:59)
1732901963 W BUY 1.70915681 cbBTC of 166123.143134 USDC (at 2024-11-29 17:39:23)
1732902095 W BUY 1.70905783 cbBTC of 166109.252964 USDC (at 2024-11-29 17:41:35)
1732902167 W BUY 0.67945462 cbBTC of 66038.819372 USDC (at 2024-11-29 17:42:47)
1732903103 W BUY 0.67951351 cbBTC of 66045.035133 USDC (at 2024-11-29 17:58:23)
1732903235 W BUY 3.77017004 cbBTC of 366365.845287 USDC (at 2024-11-29 18:00:35)
1732903259 W BUY 1.70911485 cbBTC of 166110.454259 USDC (at 2024-11-29 18:00:59)
1732903283 W BUY 1.709472 cbBTC of 166108.251218 USDC (at 2024-11-29 18:01:23)
1732903487 W BUY 0.68001354 cbBTC of 66096.947235 USDC (at 2024-11-29 18:04:47)
1732903511 W BUY 0.68014066 cbBTC of 66101.175079 USDC (at 2024-11-29 18:05:11)
1732903679 W BUY 0.16485178 cbBTC of 16021.793511 USDC (at 2024-11-29 18:07:59)
1732903727

In [ ]:
if not record_df_1.empty:
    unique_tokens_count = record_df_1["token_symbol"].nunique()
    unique_tokens = record_df_1["token_symbol"].unique()
    results = []

    for token in unique_tokens:
        token_df = record_df_1[record_df_1["formatted_record"].str.contains(fr"\b{token}\b", regex=True)]
        print(f"{token}: {len(token_df)} records")

        pnl_df = calculate_pnl(token_df)
        print(tabulate(pnl_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))   
        total_pnl = pnl_df["pnl"].sum()

        results.append({"token": token, "total_pnl": total_pnl})

    pnl_summary_df = pd.DataFrame(results)

    print(tabulate(pnl_summary_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))
else:
    print("No transactions found")


In [ ]:
matched_records = []
matched_indices = []
transaction_data_1 = transaction_data[~transaction_data['hash'].isin(duplicate_hashes['hash'])].reset_index(drop=True)

skip_next = False
for i in range(len(transaction_data_1) - 1):
    if skip_next:
        skip_next = False
        continue

    current_row = transaction_data_1.iloc[i]
    next_row = transaction_data_1.iloc[i + 1]

    if ((current_row['to'] == ADDRESS and next_row['from'] == ADDRESS) or
        (current_row['from'] == ADDRESS and next_row['to'] == ADDRESS)):
        
        if ((current_row['tokenSymbol'] in BASE_TOKENS or next_row['tokenSymbol'] in BASE_TOKENS) and
        not (current_row['tokenSymbol'] in BASE_TOKENS and next_row['tokenSymbol'] in BASE_TOKENS)):
            
            if current_row['to'] == ADDRESS:
                transaction_type = "\'SELL\'"
                base_token = current_row
                other_token = next_row
            else:
                transaction_type = "\'BUY\'"
                base_token = next_row
                other_token = current_row

            base_token_info = f"{base_token['ActualValue']} {base_token['tokenSymbol']}"
            other_token_info = f"{other_token['ActualValue']} {other_token['tokenSymbol']}"

            record = (
                f"{current_row['timeStamp']} W {transaction_type} {other_token_info} of {base_token_info} "
                f"(at {current_row['dateTime']})"
            )
            
            matched_records.append({"formatted_record": record, "dateTime": current_row['dateTime']})
            matched_indices.extend([i, i + 1])

            skip_next = True

# delete the matched data for later processing
transaction_data_2 = transaction_data_1.drop(index=matched_indices).reset_index(drop=True)

matched_df = pd.DataFrame(matched_records)

# combine the first two dataframes and sort by time
combined_df = pd.concat([output_df, matched_df], ignore_index=True)

if not combined_df.empty:
    combined_df = combined_df.sort_values(by='dateTime').reset_index(drop=True)
    for record in combined_df['formatted_record']:
        print(record)

1727625191 Same Hash 0xbaa4ff2a1d9aa7b157b8e5288c1eda1839c50779a605c8d1d87476eba19ab935: Tokens not in BASE_TOKENS: aEthUSDT, WETH (at 2024-09-29 15:53:11)
1727625203 Same Hash 0x26cb3c31a1ce7ef83ff2c3f0ad1be41eb058ae3e2fe40800b5c741202613674b: Tokens not in BASE_TOKENS: aEthUSDT, WETH (at 2024-09-29 15:53:23)
1727625227 Same Hash 0xb87fd84880b17d7c0decedbc34e101099b64a307c299bee18c01cc63dcfa98cc: Tokens not in BASE_TOKENS: aEthUSDT, WETH (at 2024-09-29 15:53:47)
1727625239 Same Hash 0x6038ad2f0efecf0eca0d28f6e6e994eac175cfcfd9ac0792e1ad3fda8fb9ce3e: Tokens not in BASE_TOKENS: aEthUSDT, WETH (at 2024-09-29 15:53:59)
1727625251 Same Hash 0xf2a695b8576be4c71a3afc29afc45d026126bba39f3de9920126da948dd7af0f: Tokens not in BASE_TOKENS: aEthUSDT, WETH (at 2024-09-29 15:54:11)
1727625263 Same Hash 0x7a1b4fc023269ce9f1a24ffa8f44dcbc49048be03c64b9614a0bcc9ce851a2f2: Tokens not in BASE_TOKENS: aEthUSDT, WETH (at 2024-09-29 15:54:23)
1727625275 Same Hash 0xa79ffa108fe373f60a3d942ab3cee4a25d7f4e4ef

In [ ]:
single_records = []

for i, row in transaction_data_2.iterrows():
    if row['tokenSymbol'] in BASE_TOKENS:
        if row['from'] == ADDRESS:
            transaction_type = "single SELL"
        elif row['to'] == ADDRESS:
            transaction_type = "single BUY"
    else:
        if row['from'] == ADDRESS:
            transaction_type = "single BUY"
        elif row['to'] == ADDRESS:
            transaction_type = "single SELL"

    record = f"{row['timeStamp']} W {transaction_type} {row['ActualValue']} {row['tokenSymbol']} (at {row['dateTime']})"
    single_records.append({
        "formatted_record": record,
        "dateTime": row['dateTime'],
        "timeStamp": row['timeStamp'],
        "hash": row['hash']
    })

single_df = pd.DataFrame(single_records)

final_combined_df = pd.concat([output_df, matched_df, single_df], ignore_index=True)
if not final_combined_df.empty:
    final_combined_df = final_combined_df.sort_values(by='dateTime').reset_index(drop=True)
    for record in final_combined_df['formatted_record']:
        print(record)


1727573591 W single SELL 52050802583630410000000000 SHIB (at 2024-09-29 01:33:11)
1727573939 W single SELL 2979587067423 aEthUSDT (at 2024-09-29 01:38:59)
1727625191 Same Hash 0xbaa4ff2a1d9aa7b157b8e5288c1eda1839c50779a605c8d1d87476eba19ab935: Tokens not in BASE_TOKENS: aEthUSDT, WETH (at 2024-09-29 15:53:11)
1727625203 Same Hash 0x26cb3c31a1ce7ef83ff2c3f0ad1be41eb058ae3e2fe40800b5c741202613674b: Tokens not in BASE_TOKENS: aEthUSDT, WETH (at 2024-09-29 15:53:23)
1727625227 Same Hash 0xb87fd84880b17d7c0decedbc34e101099b64a307c299bee18c01cc63dcfa98cc: Tokens not in BASE_TOKENS: aEthUSDT, WETH (at 2024-09-29 15:53:47)
1727625239 Same Hash 0x6038ad2f0efecf0eca0d28f6e6e994eac175cfcfd9ac0792e1ad3fda8fb9ce3e: Tokens not in BASE_TOKENS: aEthUSDT, WETH (at 2024-09-29 15:53:59)
1727625251 Same Hash 0xf2a695b8576be4c71a3afc29afc45d026126bba39f3de9920126da948dd7af0f: Tokens not in BASE_TOKENS: aEthUSDT, WETH (at 2024-09-29 15:54:11)
1727625263 Same Hash 0x7a1b4fc023269ce9f1a24ffa8f44dcbc49048be03c